In [33]:
import os
import json
import numpy as np
from sklearn.metrics import v_measure_score
from sklearn.metrics import adjusted_rand_score
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [34]:
DATA_DIR     = '../data/lsir'
LABELS_DIR   = 'labels'
CLUSTERS_DIR = 'clusters'

In [3]:
# with open(os.path.join(DATA_DIR, 'labels_true.json'), mode='r') as fp:
#     labels_true = json.load(fp)


# names = []
# x = []
# y = []
    
# for subdir, dirs, files in os.walk(os.path.join(DATA_DIR, LABELS_DIR)):
#     for filename in tqdm(files):
#         with open(os.path.join(subdir, filename), mode='r') as fp:
#             labels_pred = json.load(fp)
        
#         n = filename[:-5]

#         s1 = v_measure_score(labels_true, labels_pred)
#         s2 = adjusted_rand_score(labels_true, labels_pred)
        
#         names.append(n)
#         x.append(s1)
#         y.append(s2)

# names = np.array(names)
# x = np.array(x)
# y = np.array(y)


# x, y = x[(x > 0) & (y > 0)], y[(x > 0) & (y > 0)]

In [4]:
# import matplotlib.pyplot as plt
# import numpy as np; np.random.seed(1)
# %matplotlib notebook

# c = np.random.randint(1,5,size=len(x))

# norm = plt.Normalize(1,4)
# cmap = plt.cm.RdYlGn

# fig,ax = plt.subplots(figsize=(10,10))
# sc = plt.scatter(x,y,c=c, s=100, cmap=cmap, norm=norm)

# annot = ax.annotate("", xy=(0,0), xytext=(-20,-20),textcoords="offset points",
#                     bbox=dict(boxstyle="round", fc="w"),
#                     arrowprops=dict(arrowstyle="->"))
# annot.set_visible(False)

# def update_annot(ind):

#     pos = sc.get_offsets()[ind["ind"][0]]
#     annot.xy = pos
#     text = "{}, {}".format(" ".join(list(map(str,ind["ind"]))), 
#                            " ".join([names[n] for n in ind["ind"]]))
#     annot.set_text(text)
#     annot.get_bbox_patch().set_facecolor(cmap(norm(c[ind["ind"][0]])))
#     annot.get_bbox_patch().set_alpha(0.4)


# def hover(event):
#     vis = annot.get_visible()
#     if event.inaxes == ax:
#         cont, ind = sc.contains(event)
#         if cont:
#             update_annot(ind)
#             annot.set_visible(True)
#             fig.canvas.draw_idle()
#         else:
#             if vis:
#                 annot.set_visible(False)
#                 fig.canvas.draw_idle()

# fig.canvas.mpl_connect("motion_notify_event", hover)

# plt.show()

PEOPLE, PLACES, ORG

statistics for best params (cluster distribution)
    
top-k clusters (#articles, homogeniety)
    extract articles (title, text, entities)


!! where the algo succeed and failed !!

run on our dataset


In [35]:
import scipy.sparse as sp


with open(os.path.join(DATA_DIR, 'articles.json'), mode='r') as fp:
    articles = np.array(json.load(fp))


with open(os.path.join(DATA_DIR, CLUSTERS_DIR, 'a=0.70 b=0.30 c=0.00 thr=0.70.json'), mode='r') as fp:
    n, cluster_distr, article_distr, cluster_centers_3 = json.load(fp).values()
    vocab, data, indices, indptr = cluster_centers_3.values()

    n             = np.array(n, dtype=np.int)
    cluster_distr = dict(cluster_distr)
    article_distr = dict(article_distr)
    
    vocab     = dict(vocab)
    centers   = sp.csr_matrix((data, indices, indptr), (len(n), len(vocab)), dtype=np.float)

    vocab_inv = { i: term for term, i in vocab.items() }

In [36]:
from collections import Counter
from operator import itemgetter

sorted(Counter(n).items(), key=itemgetter(0), reverse=True)

[(70, 1),
 (56, 1),
 (52, 1),
 (50, 1),
 (41, 1),
 (40, 1),
 (32, 1),
 (30, 1),
 (27, 1),
 (23, 1),
 (22, 1),
 (21, 1),
 (20, 2),
 (19, 3),
 (18, 1),
 (16, 1),
 (15, 2),
 (14, 1),
 (13, 3),
 (12, 6),
 (11, 5),
 (10, 7),
 (9, 11),
 (8, 14),
 (7, 25),
 (6, 21),
 (5, 41),
 (4, 74),
 (3, 202),
 (2, 665),
 (1, 6176)]

In [43]:
from operator import itemgetter
c = np.where(n == 9)[0]
print(', '.join(list(map(str, c[:10]))) + ' ... ' + f'total: {len(c)}')
c = c[0]
t = np.argsort(-centers[c].A[0])[:10].tolist()
print(c, t)
itemgetter(*t)(vocab_inv)

57, 60, 247, 298, 337, 340, 1050, 1251, 1306, 2752 ... total: 11
57 [56692, 134, 102, 6132, 19522, 19523, 133, 132, 4914, 2138]


('fauci',
 'virus',
 'johnson',
 'variant',
 'debt cancellation',
 'generation ivy league graduate',
 'vaccines',
 'vaccine',
 'black',
 'wrong')

In [44]:
cluster_distr[c]

[57, 406, 2904, 4166, 5614, 5636, 5717, 6999, 7116]

In [45]:
list(map(itemgetter('title'), articles[cluster_distr[c]]))

["Analysis: Fauci's Messaging on Vaccines Is Counter-Productive and Demoralizing",
 "What Dr. Fauci is doing now that he's been vaccinated",
 "Dr. Fauci says new data suggest 'long' Covid symptoms can last up to 9 months",
 'Fauci urges public to take any COVID-19 vaccine',
 "What Dr. Fauci is doing now that he's been vaccinated",
 'Dr. Fauci: Whatever COVID-19 vaccine is available, take it',
 'Fauci: Whatever COVID vaccine is available, take it',
 'At the double! Vaccines on a roll as Fauci says jabs will end pandemic',
 'Whatever coronavirus vaccine is available, take it, Fauci says']

In [46]:
list(map(itemgetter('text'), articles[cluster_distr[c]]))

['If you listen to some skeptics of the COVID-19 vaccines, you\'ll hear various doubts about effectiveness and safety, which have not been validated by the data. To the contrary, the vaccines are working miraculously well and have been proven safe among many millions of recipients across the world. Another argument that certain fence-sitters may advance is that the government isn\'t eager to give up control over people\'s lives anyway, so why bother getting a set of injections against a virus that probably won\'t kill you -- especially if the Powers That Be won\'t allow your life to return to normal as a reward. I\'m concerned that this latter concern is being vindicated by rhetoric from top officials and amplified in the media. Reaching herd immunity is a crucial goal that will allow American life to move past the pandemic. Incessantly telling people that getting inoculated is not an effective ticket back to freedom and normalcy is a mistake. A significant number of people might well 

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

In [15]:
x = torch.rand(10,5)
x[None,:,:].shape, x[:,None,:].shape, x[:,:,None].shape

(torch.Size([1, 10, 5]), torch.Size([10, 1, 5]), torch.Size([10, 5, 1]))

In [16]:
x = torch.tensor([1.]).requires_grad_()
z = 1/x
torch.autograd.grad(z, x, retain_graph = True)
torch.autograd.grad(z * z, x)

(tensor([-2.]),)

## Questions

[handout-2-5](https://fleuret.org/dlc/materials/dlc-handout-2-5-basic-embeddings.pdf)

Slide 10, Why $x_n^{(t-1)}$ if we only do projection?

[handout-3-1](https://fleuret.org/dlc/materials/dlc-handout-3-1-perceptron.pdf)

Slide 14, Shouldn't it be $w \cdot x + b = 0$

[handout-3-5](https://fleuret.org/dlc/materials/dlc-handout-3-5-gradient-descent.pdf)

Slide 4, where is the Jaccobian term?

[handout-6-1](https://fleuret.org/dlc/materials/dlc-handout-6-1-benefits-of-depth.pdf)

Slide 8, why $\frac{1}{2}$ after the $\ge$?

[handout-6-5](https://fleuret.org/dlc/materials/dlc-handout-6-5-residual-networks.pdf)

Slide 10, shouldn't it be $C \times 1 \times 1$?
